# **Introduction**

## **Chi-test (boroughs + svi)**
## **Bar-chart with svi as regression/scatterplot (zipcode)**

source: https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documezipcodetion.pdf

source: https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

Chi-test: Using the average_year_eviction_zipcode_count for Chi-square test for zipcode, we get results as following:

1) SVI: P-value is 0.0000 and chi-square is 31.25. We can reject the null hypothesis. The Chi-square suggests a strong-to-moderate deviation from the distribution if the two groups/categories are independent. The odds are 83.31, meaning if you live in a zipcode with higher than average svi, you are 83.31 times more likely to be evicted than living in a zipcode with less than average svi.

2) Black + Hispanic Population percezipcodege: P-value is 0.0000 and chi-squiare is 72.40. We can refute the null hypothesis. We can say we are confident that high "black+hispanic percetage" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high BH pct and high eviction rates in the full dataset. The odds are 33.49, meaning if you live in a zipcode with b+h population above the average pct of nyc, you are 12.5125 times more likely to get evicted than someone living in a zipcode with below average percezipcodege of black + hispanic people.

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy esda splot

In [159]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
from scipy.stats import chi2_contingency
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [51]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Step 1 Get the Eviction data**

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
# data source:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_svi_zipcode.csv'
file_path2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_svi_zipcode.csv'

In [92]:
evictions_pre_post_raw = pd.read_csv(file_path1)
evictions_covid_raw = pd.read_csv(file_path2)
evictions_covid_raw.shape, evictions_pre_post_raw.shape

((5386, 120), (66395, 121))

In [93]:
evictions_pre_post_raw.isna().sum().sum()

np.int64(0)

In [94]:
nan_rows = evictions_pre_post_raw[evictions_pre_post_raw.isna().any(axis=1)]  #
nan_rows[['zipcode']]

,zipcode


In [95]:
evictions_pre_post = evictions_pre_post_raw.copy()
evictions_covid = evictions_covid_raw.copy()

In [96]:
evictions_pre_post.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_5year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z),hburd_impacted_zipcode_per_1kpop(z)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.6721,-73.8911,5.0000,37.0000,1152.0000,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),1930.0000,C0,3.0000,3.0000,356 MILLER LLC,2700.0000,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0000,0.9788,0.9140,0.9808,0.9812,0.9839,33.9000,11.1000,19.1000,6.0000,13.8000,22.5000,13.8000,5.3000,57.8000,9.1000,44.7000,55.9000,32.8000,1.5000,0.0000,0.0000,2.9000,1.6000,94.7000,5.3000,False,Q3,medium-high,0.0000,0.0000,1.0000,0.0000,1.0000,2.0000,0.0000,0.0000,1.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,0.0000,3.0000,5.0000,19.0000,0.2667,0.8000,266.0000,2.7479,725,2175.0000,1.2634,0.6183,0.9315,0.3792,0.3050,0.3792,2.4374,1.2283,3.6690,1.3333,88.7000,30749.0000,9431.0000,1320,0.1400,13.6362,0.0028,3.0682,4.6227,1.8818,1.5136,1.8818,12.0953,6.0954
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.6359,-74.0119,7.0000,38.0000,118.0000,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),1920.0000,B2,2.0000,2.0000,"A.R.M. PARKING, LLC",1204.0000,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0000,0.9885,0.7635,0.9594,0.9179,0.9662,37.5000,7.5000,37.9000,11.6000,13.1000,25.4000,8.4000,40.2000,61.7000,23.7000,43.6000,1.7000,40.9000,40.7000,0.4000,0.0000,1.2000,0.2000,85.0000,15.0000,False,Q3,medium-high,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.000

In [97]:
evictions_covid.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1kpop(nta),buildings_affected_per_nta,unit_per_nta,evictions_per_1kunit_nta_1kpop(nta),children_impacted_nta_per_1kpop,poverty_impacted_nta_per_1kpop,disability_impacted_nta_per_1kpop,unemployed_impacted_nta_per_1kpop,elderly_impacted_nta_per_1kpop,bh_impacted_nta_per_1kpop,hburd_impacted_nta_per_1kpop,average_year_eviction_per_building_nta(nta),average_3year_eviction_count_per_unit(building),bh_pct,unit_count_zipcode,building_count_zipcode,average_year_eviction_count_per_zipcode(z),average_year_eviction_per_building_zipcode(z),evictions_per_zipcode_1kpop(z),evictions_per_1kunit_zipcode_1kpop(z),children_impacted_zipcode_per_1kpop(z),poverty_impacted_zipcode_per_1kpop(z),disability_impacted_zipcode_per_1kpop(z),unemployed_impacted_zipcode_per_1kpop(z),elderly_impacted_zipcode_per_1kpop(z),bh_impacted_zipcode_per_1kpop(z),hburd_impacted_zipcode_per_1kpop(z)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.8654,-73.9013,7.0000,14.0000,265.0000,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),2004.0000,C0,3.0000,3.0000,MONJU SARKER,3420.0000,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0000,0.9954,0.9407,0.9870,0.9470,0.9874,39.5000,11.6000,28.3000,9.2000,11.2000,26.4000,12.2000,26.9000,71.8000,19.2000,56.7000,15.6000,78.0000,2.3000,0.0000,0.0000,0.5000,0.5000,96.9000,3.1000,False,Q3,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,2.0000,0.0000,0.0000,0.0000,3.0000,1.0000,14.0000,0.0667,0.2000,18.6667,0.2293,38,114.0000,2.0117,0.0605,0.0906,0.0280,0.0266,0.0257,0.2147,0.1300,4.9123,0.2000,93.6000,26048.0000,1266.0000,124,0.0979,1.5234,0.0008,0.4022,0.6017,0.1859,0.1767,0.1706,1.4259,0.8638
1,0050153/20_106030,4031560133,0050153/20,106030,98-05 67TH AVENUE,12F,2022-04-14,QUEENS,11375,Not an Ejectment,Possession,40.7242,-73.8556,6.0000,29.0000,71306.0000,4074666,Forest Hills,2022,2022-04,POINT (-73.855552 40.724241),1960.0000,D3,13.0000,181.0000,MARSEILLES LEASING LIMITED PARTNERSHIP,177710.0000,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternativ...","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,11375,75212.0000,0.4759,0.5698,0.8789,0.8057,0.7322,12.0000,4.8000,6.1000,3.7000,20.4000,18.0000,10.5000,7.9000,41.9000,5.8000,25.4000,2.7000,16.4000,28.5000,0.1000,0.0000,4.6000,0.7000,53.0000,47.0000,False,Q1 (Low),0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,0.0000,62.0000,0.0000,34.00

In [98]:
evictions_pre_post.columns, \
evictions_covid.columns, \
evictions_pre_post.shape, \
evictions_covid.shape

(Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment',
        ...
        'average_year_eviction_per_building_zipcode(z)',
        'evictions_per_zipcode_1kpop(z)',
        'evictions_per_1kunit_zipcode_1kpop(z)',
        'children_impacted_zipcode_per_1kpop(z)',
        'poverty_impacted_zipcode_per_1kpop(z)',
        'disability_impacted_zipcode_per_1kpop(z)',
        'unemployed_impacted_zipcode_per_1kpop(z)',
        'elderly_impacted_zipcode_per_1kpop(z)',
        'bh_impacted_zipcode_per_1kpop(z)',
        'hburd_impacted_zipcode_per_1kpop(z)'],
       dtype='object', length=121),
 Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment',
        ...
        'average_year_eviction_per_building_zipcode(z)',
        'evictions_

In [99]:
link = '/content/drive/My Drive/X999/svi_cleaned.csv'

In [100]:
svi_df = pd.read_csv(link)
svi_df.head(2)

,fips,location,area_sqmi,e_totpop,m_totpop,e_hu,m_hu,e_hh,m_hh,e_pov150,m_pov150,e_unemp,m_unemp,e_hburd,m_hburd,e_nohsdp,m_nohsdp,e_uninsur,m_uninsur,e_age65,m_age65,e_age17,m_age17,e_disabl,m_disabl,e_sngpnt,m_sngpnt,e_limeng,m_limeng,e_minrty,m_minrty,e_munit,m_munit,e_mobile,m_mobile,e_crowd,m_crowd,e_noveh,m_noveh,e_groupq,m_groupq,ep_pov150,mp_pov150,ep_unemp,mp_unemp,ep_hburd,mp_hburd,ep_nohsdp,mp_nohsdp,ep_uninsur,mp_uninsur,ep_age65,mp_age65,ep_age17,mp_age17,ep_disabl,mp_disabl,ep_sngpnt,mp_sngpnt,ep_limeng,mp_limeng,ep_minrty,mp_minrty,ep_munit,mp_munit,ep_mobile,mp_mobile,ep_crowd,mp_crowd,ep_noveh,mp_noveh,ep_groupq,mp_groupq,epl_pov150,epl_unemp,epl_hburd,epl_nohsdp,epl_uninsur,spl_theme1,rpl_theme1,epl_age65,epl_age17,epl_disabl,epl_sngpnt,epl_limeng,spl_theme2,rpl_theme2,epl_minrty,spl_theme3,rpl_theme3,epl_munit,epl_mobile,epl_crowd,epl_noveh,epl_groupq,spl_theme4,rpl_theme4,spl_themes,rpl_themes,f_pov150,f_unemp,f_hburd,f_nohsdp,f_uninsur,f_theme1,f_age65,f_age17,f_disabl,f_sngpnt,f_limeng,f_theme2,f_minrty,f_theme3,f_munit,f_mobile,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_daypop,e_noint,m_noint,e_afam,m_afam,e_hisp,m_hisp,e_asian,m_asian,e_aian,m_aian,e_nhpi,m_nhpi,e_twomore,m_twomore,e_otherrace,m_otherrace,ep_noint,mp_noint,ep_afam,mp_afam,ep_hisp,mp_hisp,ep_asian,mp_asian,ep_aian,mp_aian,ep_nhpi,mp_nhpi,ep_twomore,mp_twomore,ep_otherrace,mp_otherrace
0,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000
1,10002,ZCTA5 10002,0.8223,76518,2894,39094,1241,36028,1326,27908,2853,2833,574,14688,1367,18301,1376,4074,766,17681,1287,10028,1549,9896,1062,2211,499,18393,1640,56964,3226,35725,1677,16,28,2461,449,29828,1403,2090,39,36.8000,3.5000,7.6000,1.4000,40.8000,3.5000,30.0000,2.0000,5.4000,1.0000,23.1000,1.7000,13.1000,1.8000,13.0000,1.4000,6.1000,1.4000,24.7000,2.0000,74.4000,3.1000,91.4000,3.2000,0.0000,0.1000,6.8000,1.2000,82.8000,1.8000,2.7000,0.1000,0.9148,0.7946,0.9219,0.9741,0.7207,4.3261,0.9639,0.7296,0.1831,0.5186,0.7390,0.9944,3.1647,0.8781,0.9369,0.9369,0.9369,0.9790,0.0000,0.9105,0.9915,0.7730,3.6540,0.9254,12.0817,0.9656,1,0,1,1,0,3,0,0,0,0,1,1,1,1,1,0,1,1,0,3,8,64307,8590,1110,6141,1194,19864,2190,28477,1989,74,83,24,45,1810,486,574,394,23.8000,2.9000,8.0000,1.5000,26.0000,2.5000,37.2000,2.2000,0.1000,0.1000,0.0000,0.1000,2.4000,0.6000,0.8000,0.5000


In [101]:
svi_df.shape

(204, 153)

In [102]:
# list(svi_df.columns)

In [103]:
svi_df.ep_nhpi.unique()

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

# **Step 2: SVI items**

q quick double check

In [104]:
link = "/content/drive/My Drive/X999/NewYork_ZCTA.csv"

In [105]:
svi_raw = pd.read_csv(link)
svi_raw.head(2)

,ST,STATE,ST_ABBR,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,36,New York,NY,6390,ZCTA5 06390,4.0467,53,39,253,49,19,19,17,16,0,13,9,26,0,13,27,34,0,13,6,11,31,33,0,18,9,53,20,51,0,18,4,5,0,18,0,13,17,16,32.1000,18.8000,0.0000,52.7000,47.4000,100.0000,0.0000,51.4000,50.9000,45.7000,0.0000,45.2000,11.3000,19.0000,58.5000,41.4000,0.0000,94.7000,17.0000,99.2000,37.7000,92.1000,0.0000,7.1000,1.6000,2.0000,0.0000,94.7000,0.0000,75.5000,32.1000,18.8000,0.8790,0.0000,0.9635,0.0000,0.9960,2.8385,0.6342,0.0000,0.1408,0.9944,0.0000,0.9775,2.1127,0.3009,0.8062,0.8062,0.8062,0.0000,0.4654,0.0000,0.0000,0.9735,1.4389,0.2205,7.1963,0.4192,0,0,1,0,1,2,0,0,1,0,1,2,0,0,0,0,0,0,1,1,5,601,9,14,0,13,9,19,0,13,0,13,8,16,3,7,0,13,47.4000,51.8000,0.0000,45.2000,17.0000,35.0000,0.0000,45.2000,0.0000,45.2000,15.1000,32.1000,5.7000,12.1000,0.0000,45.2000
1,36,New York,NY,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000


In [106]:
def is_nyc_zipcode(zipcode):
    zip_int = int(zipcode) if isinstance(zipcode, str) else zipcode

    # Manhattan: 10001-10282
    if 10001 <= zip_int <= 10282:
        return True
    # addition Manhattan: 10300-10499
    if 10300 <= zip_int <= 10499:
        return True
    # Bronx: 10451-10475
    if 10451 <= zip_int <= 10475:
        return True
    # Brooklyn: 11201-11256
    if 11201 <= zip_int <= 11256:
        return True
    # Queens: 11351-11436, 11101-11109
    if (11351 <= zip_int <= 11436) or (11101 <= zip_int <= 11109):
        return True
    # Staten Island: 10301-10314
    if 10301 <= zip_int <= 10314:
        return True
    # additional Queens ZIPs
    if zip_int in [11004, 11005, 11411, 11412, 11413, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429]:
        return True
    return False

In [107]:
nyc_df = svi_raw[svi_raw['FIPS'].apply(is_nyc_zipcode)]

In [108]:
nyc_df.shape

(204, 156)

In [109]:
nyc_df.EP_NHPI.unique()
# interesting, so there was an error in the source data

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

In [110]:
svi_raw.EP_NHPI.unique()

array([ 1.51e+01,  0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,
        8.00e-01,  1.20e+00,  5.00e-01,  4.00e-01,  1.10e+00,  7.00e-01,
        1.50e+00,  1.80e+00,  9.00e-01,  6.00e-01,  1.40e+00,  2.10e+00,
        2.30e+00,  1.00e+00])

In [111]:
-9.99e+02, 0.00e+00, 9.00e-01, 2.00e-01

(-999.0, 0.0, 0.9, 0.2)

# **Step 3: All boroughs and their eviction rates**

In [112]:
evictions_pre_post_mean = evictions_pre_post[['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']].mean()
evictions_pre_post_mean
# may need to merge ep_twomore and ep_otherrance toegther

,0
ep_afam,29.2498
ep_asian,8.9193
ep_hisp,38.7465
ep_nhpi,0.0265
ep_white,19.0073
ep_twomore,2.8176
ep_otherrace,1.0390


In [113]:
evictions_pre_post_mean = evictions_pre_post_mean.reset_index()

In [114]:
evictions_pre_post_mean.rename(columns = {'index':'race_svi', 0: "racial percezipcodege"}, inplace=True)

In [115]:
evictions_pre_post_mean

,race_svi,racial percezipcodege
0,ep_afam,29.2498
1,ep_asian,8.9193
2,ep_hisp,38.7465
3,ep_nhpi,0.0265
4,ep_white,19.0073
5,ep_twomore,2.8176
6,ep_otherrace,1.0390


In [116]:
# type(evictions_pre_post_mean)
# so this is correct

In [117]:
# evictions_pre_post.columns

In [118]:
evictions_pre_post.columns[-30:]

Index(['average_year_eviction_count_per_unit(building)',
       'average_year_eviction_count(building)',
       'average_year_eviction_count_per_nta(nta)',
       'evictions_per_nta_1kpop(nta)', 'buildings_affected_per_nta',
       'unit_per_nta', 'evictions_per_1kunit_nta_1kpop(nta)',
       'children_impacted_nta_per_1kpop', 'poverty_impacted_nta_per_1kpop',
       'disability_impacted_nta_per_1kpop',
       'unemployed_impacted_nta_per_1kpop', 'elderly_impacted_nta_per_1kpop',
       'bh_impacted_nta_per_1kpop', 'hburd_impacted_nta_per_1kpop',
       'average_year_eviction_per_building_nta(nta)',
       'average_5year_eviction_count_per_unit(building)', 'bh_pct',
       'unit_count_zipcode', 'building_count_zipcode',
       'average_year_eviction_count_per_zipcode(z)',
       'average_year_eviction_per_building_zipcode(z)',
       'evictions_per_zipcode_1kpop(z)',
       'evictions_per_1kunit_zipcode_1kpop(z)',
       'children_impacted_zipcode_per_1kpop(z)',
       'poverty_impacte

In [121]:
evictions_pre_post['average_year_eviction_per_building_zipcode(z)'].mean()
# per building, per year, that's why it is similar to the borough one too

np.float64(0.46716589822318577)

In [122]:
evictions_pre_post['average_year_eviction_count_per_zipcode(z)'].mean()

np.float64(909.649898335718)

### **Step 3.2. All zipcodes racial composite**

In [137]:
race_columns = ['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']
racial_avg_all_zt = evictions_pre_post.groupby('zipcode')[['average_year_eviction_count_per_zipcode(z)', 'ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace', 'rpl_themes']].mean()
racial_avg_all_zt.sort_values('average_year_eviction_count_per_zipcode(z)', ascending=False, inplace=True)
racial_avg_all_zt.reset_index(inplace=True)
racial_avg_all_zt

,zipcode,average_year_eviction_count_per_zipcode(z),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes
0,10458,2221.0000,16.1000,3.3000,71.3000,0.0000,7.0000,1.7000,0.4000,0.9891
1,10467,2192.0000,28.6000,6.2000,52.9000,0.0000,8.7000,2.3000,0.7000,0.9925
2,10453,1994.0000,24.6000,1.6000,69.2000,0.0000,2.0000,1.9000,0.7000,0.9983
3,10456,1880.0000,38.2000,0.7000,56.3000,0.0000,2.4000,1.8000,0.4000,0.9994
4,10457,1765.0000,30.9000,0.4000,65.3000,0.0000,1.7000,1.1000,0.4000,0.9989
5,10468,1719.0000,15.6000,2.3000,78.0000,0.0000,3.1000,0.5000,0.5000,0.9874
6,11226,1639.0000,63.2000,3.2000,14.9000,0.0000,13.7000,4.1000,0.7000,0.9220
7,10452,1592.0000,28.1000,1.0000,65.4000,0.0000,2.7000,1.4000,1.1000,0.9931
8,10460,1557.0000,26.5000,1.4000,65.0000,0.0000,3.8000,1.9000,1.0000,0.9954
9,11212,1487.0000,70.1000,0.8000,19.5000,0.0000,3.5000,5.4000,0.6000,0.9948


In [138]:
racial_avg_all_zt.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173
zipcode,10458.0000,10467.0000,10453.0000,10456.0000,10457.0000,10468.0000,11226.0000,10452.0000,10460.0000,11212.0000,11207.0000,11233.0000,11208.0000,10466.0000,10472.0000,10459.0000,11213.0000,11203.0000,10451.0000,10462.0000,11221.0000,11210.0000,11368.0000,10463.0000,10029.0000,10032.0000,11225.0000,10031.0000,10455.0000,10033.0000,11236.0000,10469.0000,10034.0000,11216.0000,10454.0000,10030.0000,10473.0000,11206.0000,10027.0000,10040.0000,11230.0000,10039.0000,10037.0000,11434.0000,10035.0000,11235.0000,11432.0000,10461.0000,10301.0000,10304.0000,11224.0000,11372.0000,11435.0000,11385.0000,11214.0000,10026.0000,11373.0000,11209.0000,10025.0000,11377.0000,10019.0000,10009.0000,11433.0000,11354.0000,11223.0000,11238.0000,11220.0000,11229.0000,10475.0000,10002.0000,10474.0000,11355.0000,11234.0000,11218.0000,10470.0000,11237.0000,11205.0000,11423.0000,10036.0000,11375.0000,11367.0000,11365.0000,11374.0000,10303.0000,11239.0000,10128.0000,11211.0000,11204.0000,11412.0000,10016.0000,11413.0000,11420.0000,11418.0000,11101.0000,11106.0000,11219.0000,11419.0000,11415.0000,10310.0000,10306.0000,10465.0000,11421.0000,10023.0000,10001.0000,11104.0000,11103.0000,11217.0000,11102.0000,10011.0000,10314.0000,10471.0000,10028.0000,10024.0000,11105.0000,10003.0000,11201.0000,11416.0000,11422.0000,11436.0000,10065.0000,10021.0000,10022.0000,10305.0000,10302.0000,11231.0000,11417.0000,11429.0000,11249.0000,11369.0000,11427.0000,11222.0000,10014.0000,10012.0000,11356.0000,11215.0000,10010.0000,11428.0000,11358.0000,10013.0000,11228.0000,11232.0000,11378.0000,10075.0000,10017.0000,10018.0000,10038.0000,11370.0000,11414.0000,11379.0000,11411.0000,11357.0000,10005.0000,11360.0000,11364.0000,11109.0000,11361.0000,10312.0000,10309.0000,10280.0000,11426.0000,10308.0000,10464.0000,10307.0000,11366.0000,11363.0000,10069.0000,10006.0000,11004.0000,10007.0000,11362.0000,10282.0000,10162.0000,10044.0000,11005.0000
average_year_eviction_count_per_zipcode(z),2221.0000,2192.0000,1994.0000,1880.0000,1765.0000,1719.0000,1639.0000,1592.0000,1557.0000,1487.0000,1320.0000,1196.0000,1129.0000,1119.0000,1070.0000,1041.0000,1011.0000,952.0000,932.0000,922.0000,897.0000,843.0000,810.0000,746.0000,724.0000,689.0000,682.0000,678.0000,664.0000,627.0000,612.0000,602.0000,579.0000,559.0000,546.0000,546.0000,544.0000,543.0000,532.0000,513.0000,513.0000,506.0000,506.0000,494.0000,470.0000,470.0000,468.0000,448.0000,445.0000,442.0000,440.0000,413.0000,412.0000,395.0000,387.0000,383.0000,370.0000,369.0000,349.0000,347.0000,346.0000,335.0000,325.0000,322.0000,320.0000,320.0000,317.0000,308.0000,305.0000,305.0000,297.0000,293.0000,288.0000,275.0000,267.0000,262.0000,251.0000,244.0000,242.0000,240.0000,240.0000,227.0000,225.0000,223.0000,221.0000,221.0000,218.0000,206.0000,203.0000,200.0000,199.0000,195.0000,192.0000,189.0000,186.0000,186.0000,181.0000,178.0000,165.0000,164.0000,163.0000,163.0000,158.0000,158.0000,157.0000,157.0000,156.0000,154.0000,154.0000,152.0000,151.0000,148.0000,145.0000,144.0000,138.0000,136.0000,134.0000,125.0000,124.0000,124.0000,123.0000,123.0000,120.0000,117.0000,117.0000,116.0000,114.0000,113.0000,112.0000,107.0000,104.0000,99.0000,96.0000,86.0000,84.0000,83.0000,83.0000,82.0000,80.0000,80.0000,79.0000,76.0000,75.0000,66.0000,64.0000,63.0000,63.0000,60.0000,55.0000,55.0000,53.0000,48.0000,47.0000,45.0000,43.0000,42.0000,37.0000,32.0000,31.0000,27.0000,23.0000,22.0000,20.0

In [142]:
race_columns = ['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']
racial_avg_all_zp = evictions_pre_post.groupby('zipcode')[['evictions_per_zipcode_1kpop(z)', 'ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace', 'rpl_themes']].mean()
racial_avg_all_zp.sort_values('evictions_per_zipcode_1kpop(z)', ascending=False, inplace=True)
racial_avg_all_zp.reset_index(inplace=True)
racial_avg_all_zp

# 'evictions_per_zipcode_1kpop(z)',
# 'evictions_per_1kunit_zipcode_1kpop(z)'

,zipcode,evictions_per_zipcode_1kpop(z),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes
0,10458,26.8633,16.1000,3.3000,71.3000,0.0000,7.0000,1.7000,0.4000,0.9891
1,10037,26.4395,63.1000,3.8000,19.8000,0.0000,9.6000,2.9000,0.8000,0.9639
2,10474,26.3298,26.4000,0.2000,70.2000,0.0000,2.2000,0.3000,0.2000,0.9960
3,10460,26.0142,26.5000,1.4000,65.0000,0.0000,3.8000,1.9000,1.0000,0.9954
4,10453,24.8056,24.6000,1.6000,69.2000,0.0000,2.0000,1.9000,0.7000,0.9983
5,10467,22.2058,28.6000,6.2000,52.9000,0.0000,8.7000,2.3000,0.7000,0.9925
6,10457,22.1131,30.9000,0.4000,65.3000,0.0000,1.7000,1.1000,0.4000,0.9989
7,10456,21.2250,38.2000,0.7000,56.3000,0.0000,2.4000,1.8000,0.4000,0.9994
8,10468,21.1187,15.6000,2.3000,78.0000,0.0000,3.1000,0.5000,0.5000,0.9874
9,10452,20.2483,28.1000,1.0000,65.4000,0.0000,2.7000,1.4000,1.1000,0.9931


In [143]:
racial_avg_all_zp.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173
zipcode,10458.0000,10037.0000,10474.0000,10460.0000,10453.0000,10467.0000,10457.0000,10456.0000,10468.0000,10452.0000,10459.0000,10451.0000,11212.0000,10030.0000,10470.0000,10039.0000,10472.0000,11226.0000,10466.0000,10455.0000,11213.0000,11233.0000,10034.0000,11207.0000,10454.0000,11210.0000,11239.0000,10035.0000,11203.0000,10462.0000,10040.0000,11225.0000,10032.0000,10031.0000,10301.0000,10033.0000,10463.0000,11208.0000,10026.0000,10304.0000,11221.0000,10029.0000,11224.0000,10473.0000,11216.0000,10461.0000,11415.0000,10469.0000,11433.0000,10303.0000,11423.0000,10019.0000,10027.0000,10036.0000,10018.0000,11432.0000,11434.0000,11368.0000,10475.0000,11435.0000,10471.0000,10302.0000,10310.0000,11372.0000,11236.0000,11206.0000,11354.0000,11109.0000,10001.0000,11104.0000,11436.0000,10009.0000,11102.0000,11230.0000,11235.0000,11237.0000,11367.0000,11101.0000,11238.0000,10005.0000,11209.0000,10464.0000,11418.0000,11412.0000,11205.0000,11365.0000,11416.0000,11374.0000,11106.0000,10017.0000,10012.0000,11103.0000,11413.0000,11214.0000,11427.0000,11428.0000,11429.0000,10065.0000,10006.0000,11223.0000,11377.0000,10002.0000,11421.0000,10128.0000,11422.0000,11420.0000,11229.0000,11419.0000,11218.0000,11385.0000,11417.0000,11105.0000,11355.0000,10016.0000,10022.0000,11217.0000,11373.0000,10025.0000,10465.0000,10280.0000,11369.0000,11220.0000,10014.0000,11211.0000,10028.0000,11234.0000,11375.0000,10075.0000,11356.0000,10011.0000,10021.0000,11231.0000,10306.0000,10305.0000,11232.0000,10013.0000,10038.0000,11222.0000,11411.0000,10003.0000,10010.0000,11204.0000,11249.0000,10023.0000,10069.0000,11360.0000,10024.0000,11370.0000,11358.0000,11363.0000,11219.0000,11414.0000,10162.0000,11201.0000,11378.0000,11228.0000,10314.0000,11379.0000,11361.0000,11366.0000,11426.0000,10307.0000,10007.0000,11357.0000,11364.0000,11215.0000,10309.0000,10282.0000,10308.0000,11004.0000,10312.0000,11362.0000,11005.0000,10044.0000
evictions_per_zipcode_1kpop(z),26.8633,26.4395,26.3298,26.0142,24.8056,22.2058,22.1131,21.2250,21.1187,20.2483,20.0331,18.1637,17.7011,17.4619,17.3489,16.9304,16.3902,16.2192,15.4830,14.9617,14.6244,14.3880,13.9283,13.6362,13.2934,13.2911,13.1485,12.1777,12.1265,11.9384,11.7518,11.6629,11.3922,11.1067,11.0337,10.6794,10.6123,10.4363,10.0091,9.8520,9.8316,9.5749,9.1871,9.0535,8.9622,8.8071,8.7620,8.3772,8.3623,8.2339,8.1548,8.0905,8.0844,7.8241,7.3912,7.2820,7.2453,7.1840,7.0088,6.8711,6.5973,6.3015,6.2883,6.1641,6.1150,6.0368,6.0332,5.9066,5.8510,5.8280,5.8082,5.7345,5.6892,5.6845,5.6579,5.5528,5.5445,5.5136,5.5128,5.4539,5.1384,5.1258,5.0108,5.0009,4.9882,4.8897,4.8212,4.7639,4.6271,4.5561,4.5249,4.5177,4.4388,4.2623,4.2482,4.2447,4.0941,4.0183,4.0150,3.9944,3.9896,3.9860,3.9228,3.8981,3.8704,3.8121,3.7974,3.7633,3.7508,3.7433,3.7179,3.7156,3.6858,3.6786,3.6622,3.6241,3.6056,3.5984,3.5197,3.5056,3.4807,3.4083,3.3604,3.3277,3.2373,3.2009,3.1910,3.1355,3.1123,3.0332,3.0264,3.0132,2.9165,2.8429,2.8077,2.7162,2.6989,2.5911,2.5786,2.5614,2.5609,2.5566,2.5063,2.4047,2.3965,2.3835,2.3099,2.2369,2.1781,2.1724,2.0155,1.9754,1.9698,1.9497,1.9363,1.8101,1.6109,1.4375,1.4216,1.3910,1.3683,1.3555,1.3323,1.2385,1.1804,1.1664,0.9211,0.7752,0.7648,0.7642,0.5829,0.4884,0.4598,0.1688
ep_afam,16.1000,63.1000,26.4000,26.5000,24.6000,28.6000,30.9000,38.2000,15.6000,28.1000,28.9000,37.2000,70.1000,53.8000,31.3000,54.7000,23.1000,63.2000,59.8000,21.1000,56.6000,66.7000,5.7000,55.9000,24.6

In [146]:
race_columns = ['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']
racial_avg_all_zup = evictions_pre_post.groupby('zipcode')[['evictions_per_1kunit_zipcode_1kpop(z)', 'ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace', 'rpl_themes']].mean()
racial_avg_all_zup.sort_values('evictions_per_1kunit_zipcode_1kpop(z)', ascending=False, inplace=True)
racial_avg_all_zup.reset_index(inplace=True)
racial_avg_all_zup

# 'evictions_per_zipcode_1kpop(z)',
# 'evictions_per_1kunit_zipcode_1kpop(z)'

,zipcode,evictions_per_1kunit_zipcode_1kpop(z),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes
0,10464,0.0163,0.7000,2.4000,28.9000,0.0000,66.2000,0.0000,1.7000,0.7219
1,11411,0.0122,86.8000,1.5000,6.7000,0.0000,2.0000,2.2000,0.9000,0.7001
2,11366,0.0120,4.9000,42.3000,14.0000,0.0000,33.9000,3.6000,1.4000,0.8102
3,10307,0.0091,0.1000,2.1000,10.6000,0.1000,86.0000,1.2000,0.0000,0.3624
4,11436,0.0090,61.3000,6.4000,12.9000,0.0000,1.8000,7.5000,9.5000,0.8653
5,11428,0.0073,20.0000,31.8000,26.2000,0.0000,8.4000,4.9000,8.6000,0.8853
6,10302,0.0068,21.7000,5.6000,41.9000,0.0000,28.6000,1.8000,0.4000,0.9163
7,11429,0.0064,65.4000,7.4000,15.0000,0.0000,3.0000,4.7000,4.4000,0.8807
8,11004,0.0064,5.3000,51.4000,12.2000,0.0000,25.8000,1.9000,3.3000,0.8469
9,11412,0.0060,79.9000,3.1000,7.6000,0.4000,1.3000,4.0000,3.4000,0.8383


In [145]:
racial_avg_all_zup.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173
zipcode,10458.0000,10037.0000,10474.0000,10460.0000,10453.0000,10467.0000,10457.0000,10456.0000,10468.0000,10452.0000,10459.0000,10451.0000,11212.0000,10030.0000,10470.0000,10039.0000,10472.0000,11226.0000,10466.0000,10455.0000,11213.0000,11233.0000,10034.0000,11207.0000,10454.0000,11210.0000,11239.0000,10035.0000,11203.0000,10462.0000,10040.0000,11225.0000,10032.0000,10031.0000,10301.0000,10033.0000,10463.0000,11208.0000,10026.0000,10304.0000,11221.0000,10029.0000,11224.0000,10473.0000,11216.0000,10461.0000,11415.0000,10469.0000,11433.0000,10303.0000,11423.0000,10019.0000,10027.0000,10036.0000,10018.0000,11432.0000,11434.0000,11368.0000,10475.0000,11435.0000,10471.0000,10302.0000,10310.0000,11372.0000,11236.0000,11206.0000,11354.0000,11109.0000,10001.0000,11104.0000,11436.0000,10009.0000,11102.0000,11230.0000,11235.0000,11237.0000,11367.0000,11101.0000,11238.0000,10005.0000,11209.0000,10464.0000,11418.0000,11412.0000,11205.0000,11365.0000,11416.0000,11374.0000,11106.0000,10017.0000,10012.0000,11103.0000,11413.0000,11214.0000,11427.0000,11428.0000,11429.0000,10065.0000,10006.0000,11223.0000,11377.0000,10002.0000,11421.0000,10128.0000,11422.0000,11420.0000,11229.0000,11419.0000,11218.0000,11385.0000,11417.0000,11105.0000,11355.0000,10016.0000,10022.0000,11217.0000,11373.0000,10025.0000,10465.0000,10280.0000,11369.0000,11220.0000,10014.0000,11211.0000,10028.0000,11234.0000,11375.0000,10075.0000,11356.0000,10011.0000,10021.0000,11231.0000,10306.0000,10305.0000,11232.0000,10013.0000,10038.0000,11222.0000,11411.0000,10003.0000,10010.0000,11204.0000,11249.0000,10023.0000,10069.0000,11360.0000,10024.0000,11370.0000,11358.0000,11363.0000,11219.0000,11414.0000,10162.0000,11201.0000,11378.0000,11228.0000,10314.0000,11379.0000,11361.0000,11366.0000,11426.0000,10307.0000,10007.0000,11357.0000,11364.0000,11215.0000,10309.0000,10282.0000,10308.0000,11004.0000,10312.0000,11362.0000,11005.0000,10044.0000
evictions_per_zipcode_1kpop(z),26.8633,26.4395,26.3298,26.0142,24.8056,22.2058,22.1131,21.2250,21.1187,20.2483,20.0331,18.1637,17.7011,17.4619,17.3489,16.9304,16.3902,16.2192,15.4830,14.9617,14.6244,14.3880,13.9283,13.6362,13.2934,13.2911,13.1485,12.1777,12.1265,11.9384,11.7518,11.6629,11.3922,11.1067,11.0337,10.6794,10.6123,10.4363,10.0091,9.8520,9.8316,9.5749,9.1871,9.0535,8.9622,8.8071,8.7620,8.3772,8.3623,8.2339,8.1548,8.0905,8.0844,7.8241,7.3912,7.2820,7.2453,7.1840,7.0088,6.8711,6.5973,6.3015,6.2883,6.1641,6.1150,6.0368,6.0332,5.9066,5.8510,5.8280,5.8082,5.7345,5.6892,5.6845,5.6579,5.5528,5.5445,5.5136,5.5128,5.4539,5.1384,5.1258,5.0108,5.0009,4.9882,4.8897,4.8212,4.7639,4.6271,4.5561,4.5249,4.5177,4.4388,4.2623,4.2482,4.2447,4.0941,4.0183,4.0150,3.9944,3.9896,3.9860,3.9228,3.8981,3.8704,3.8121,3.7974,3.7633,3.7508,3.7433,3.7179,3.7156,3.6858,3.6786,3.6622,3.6241,3.6056,3.5984,3.5197,3.5056,3.4807,3.4083,3.3604,3.3277,3.2373,3.2009,3.1910,3.1355,3.1123,3.0332,3.0264,3.0132,2.9165,2.8429,2.8077,2.7162,2.6989,2.5911,2.5786,2.5614,2.5609,2.5566,2.5063,2.4047,2.3965,2.3835,2.3099,2.2369,2.1781,2.1724,2.0155,1.9754,1.9698,1.9497,1.9363,1.8101,1.6109,1.4375,1.4216,1.3910,1.3683,1.3555,1.3323,1.2385,1.1804,1.1664,0.9211,0.7752,0.7648,0.7642,0.5829,0.4884,0.4598,0.1688
ep_afam,16.1000,63.1000,26.4000,26.5000,24.6000,28.6000,30.9000,38.2000,15.6000,28.1000,28.9000,37.2000,70.1000,53.8000,31.3000,54.7000,23.1000,63.2000,59.8000,21.1000,56.6000,66.7000,5.7000,55.9000,24.6

## **This is for the race composites and zipcodes bar char use**

# **Step 4 We want a dataframe that has zipcodes as columns and average_year_eviction_zipcode_count as rows and contents**

In [148]:
evi_svi_df = evictions_pre_post.groupby('zipcode')[['average_year_eviction_count_per_zipcode(z)','rpl_themes']].mean()
evi_svi_df.sort_values('average_year_eviction_count_per_zipcode(z)', ascending=False, inplace=True)
evi_svi_df

,average_year_eviction_count_per_zipcode(z),rpl_themes
zipcode,,
10458,2221.0000,0.9891
10467,2192.0000,0.9925
10453,1994.0000,0.9983
10456,1880.0000,0.9994
10457,1765.0000,0.9989
10468,1719.0000,0.9874
11226,1639.0000,0.9220
10452,1592.0000,0.9931
10460,1557.0000,0.9954


In [149]:
evi_svi_df.reset_index(inplace=True)
evi_svi_df

,zipcode,average_year_eviction_count_per_zipcode(z),rpl_themes
0,10458,2221.0000,0.9891
1,10467,2192.0000,0.9925
2,10453,1994.0000,0.9983
3,10456,1880.0000,0.9994
4,10457,1765.0000,0.9989
5,10468,1719.0000,0.9874
6,11226,1639.0000,0.9220
7,10452,1592.0000,0.9931
8,10460,1557.0000,0.9954
9,11212,1487.0000,0.9948


In [153]:
avg_per_zipcode = evi_svi_df['average_year_eviction_count_per_zipcode(z)'].mean()
avg_per_zipcode

np.float64(381.58045977011494)

In [152]:
average_svi = 0.80198
# https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

In [155]:
evi_svi_df['above_eviction_average'] = evi_svi_df['average_year_eviction_count_per_zipcode(z)'] > avg_per_zipcode
evi_svi_df

,zipcode,average_year_eviction_count_per_zipcode(z),rpl_themes,above_eviction_average
0,10458,2221.0000,0.9891,True
1,10467,2192.0000,0.9925,True
2,10453,1994.0000,0.9983,True
3,10456,1880.0000,0.9994,True
4,10457,1765.0000,0.9989,True
5,10468,1719.0000,0.9874,True
6,11226,1639.0000,0.9220,True
7,10452,1592.0000,0.9931,True
8,10460,1557.0000,0.9954,True
9,11212,1487.0000,0.9948,True


In [156]:
evi_svi_df['above_svi_average'] = evi_svi_df['rpl_themes'] > average_svi
evi_svi_df

,zipcode,average_year_eviction_count_per_zipcode(z),rpl_themes,above_eviction_average,above_svi_average
0,10458,2221.0000,0.9891,True,True
1,10467,2192.0000,0.9925,True,True
2,10453,1994.0000,0.9983,True,True
3,10456,1880.0000,0.9994,True,True
4,10457,1765.0000,0.9989,True,True
5,10468,1719.0000,0.9874,True,True
6,11226,1639.0000,0.9220,True,True
7,10452,1592.0000,0.9931,True,True
8,10460,1557.0000,0.9954,True,True
9,11212,1487.0000,0.9948,True,True


# **Step 4.3 Run Chi-Square test**
  **Null Hypothesis: There is no association between zipcodes that have above the average of black and hispanic percezipcodege and have above than average svi scores**

In [157]:
contingency_table = pd.crosstab(evi_svi_df.above_svi_average, evi_svi_df.above_eviction_average)
contingency_table

above_eviction_average,False,True
above_svi_average,,
False,50,0
True,68,56


In [167]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [163]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(31.25986599820354),
 np.float64(2.256975601346814e-08),
 1,
 array([[33.90804598, 16.09195402],
        [84.09195402, 39.90804598]]))

In [168]:
2.256975601346814e-08

2.256975601346814e-08

### **P-value is 0.0000 and chi-square is 31.25. We can reject the null hypothesis. The Chi-square suggests a strong-to-moderate deviation from the distribution if the two groups/categories are independent.**

# **conclusion: there is statistically significant association between the two "above average eviction" and "above average svi scores" zipcodes.**

In [169]:
a = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
b = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == False)].shape[0]
c = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
d = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == False)].shape[0]

In [170]:
observed = np.array([[a, b], [c, d]])
observed

array([[56, 68],
       [ 0, 50]])

In [171]:
a += 0.5
b += 0.5
c += 0.5
d += 0.5
# avoid 0 divisions

In [172]:
odds_ratio = (a * d) / (b * c)
odds_ratio
# extremely strong association between high svi and high evitction rates.

83.30656934306569

**This means, technically, zipcodes with above average svi had 83.31 times higher odds of having above average eviction rates compared to low-SVI zipcodes.** This high odds may be due to all high svi zipcodes had high evictions, and all high eviction zipcodes had high svi from the sample.

In [173]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value

(np.float64(31.259865998203537), np.float64(2.256975601346814e-08))

# **Step 5: We also need a dataframe that has zipcodes as columns and black and hispanics percezipcodeges as rows and contents**

In [174]:
racial_avg_all_zp['black_hispanic_pct'] = racial_avg_all_zp['ep_afam'] + racial_avg_all_zp['ep_hisp']
racial_avg_all_zp

,zipcode,evictions_per_zipcode_1kpop(z),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,black_hispanic_pct
0,10458,26.86,16.10,3.30,71.30,0.00,7.00,1.70,0.40,0.99,87.40
1,10037,26.44,63.10,3.80,19.80,0.00,9.60,2.90,0.80,0.96,82.90
2,10474,26.33,26.40,0.20,70.20,0.00,2.20,0.30,0.20,1.00,96.60
3,10460,26.01,26.50,1.40,65.00,0.00,3.80,1.90,1.00,1.00,91.50
4,10453,24.81,24.60,1.60,69.20,0.00,2.00,1.90,0.70,1.00,93.80
5,10467,22.21,28.60,6.20,52.90,0.00,8.70,2.30,0.70,0.99,81.50
6,10457,22.11,30.90,0.40,65.30,0.00,1.70,1.10,0.40,1.00,96.20
7,10456,21.22,38.20,0.70,56.30,0.00,2.40,1.80,0.40,1.00,94.50
8,10468,21.12,15.60,2.30,78.00,0.00,3.10,0.50,0.50,0.99,93.60
9,10452,20.25,28.10,1.00,65.40,0.00,2.70,1.40,1.10,0.99,93.50


In [177]:
racial_avg_all_zp.columns

Index(['zipcode', 'evictions_per_zipcode_1kpop(z)', 'ep_afam', 'ep_asian',
       'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace',
       'rpl_themes', 'black_hispanic_pct'],
      dtype='object')

In [178]:
evi_bh_df = racial_avg_all_zp[['zipcode', 'evictions_per_zipcode_1kpop(z)', 'black_hispanic_pct']]
evi_bh_df

,zipcode,evictions_per_zipcode_1kpop(z),black_hispanic_pct
0,10458,26.86,87.40
1,10037,26.44,82.90
2,10474,26.33,96.60
3,10460,26.01,91.50
4,10453,24.81,93.80
5,10467,22.21,81.50
6,10457,22.11,96.20
7,10456,21.22,94.50
8,10468,21.12,93.60
9,10452,20.25,93.50


In [179]:
average_bh_pct = evi_bh_df['black_hispanic_pct'].mean()
average_bh_pct

np.float64(44.84022988505747)

In [180]:
avg_per_zipcode, average_bh_pct
# evictions and black + hispanic pct

(np.float64(381.58045977011494), np.float64(44.84022988505747))

In [188]:
above_evi_1kpop_average = evi_bh_df['evictions_per_zipcode_1kpop(z)'].mean()
above_evi_1kpop_average

np.float64(6.958997919538344)

In [191]:
evi_bh_df['above_evi_average'] = evi_bh_df['evictions_per_zipcode_1kpop(z)'] > above_evi_1kpop_average
evi_bh_df

,zipcode,evictions_per_zipcode_1kpop(z),black_hispanic_pct,above_evi_average,above_bh_average
0,10458,26.86,87.40,True,True
1,10037,26.44,82.90,True,True
2,10474,26.33,96.60,True,True
3,10460,26.01,91.50,True,True
4,10453,24.81,93.80,True,True
5,10467,22.21,81.50,True,True
6,10457,22.11,96.20,True,True
7,10456,21.22,94.50,True,True
8,10468,21.12,93.60,True,True
9,10452,20.25,93.50,True,True


In [192]:
evi_bh_df['above_bh_average'] = evi_bh_df['black_hispanic_pct'] > average_bh_pct
evi_bh_df

,zipcode,evictions_per_zipcode_1kpop(z),black_hispanic_pct,above_evi_average,above_bh_average
0,10458,26.86,87.40,True,True
1,10037,26.44,82.90,True,True
2,10474,26.33,96.60,True,True
3,10460,26.01,91.50,True,True
4,10453,24.81,93.80,True,True
5,10467,22.21,81.50,True,True
6,10457,22.11,96.20,True,True
7,10456,21.22,94.50,True,True
8,10468,21.12,93.60,True,True
9,10452,20.25,93.50,True,True


### **This is the zipcodes evictions + black/hispanic chi-test df:**

# **Run Chi-Square test**
  **Null Hypothesis: There is no association between zipcodes that have above the average of black and hispanic per zipcodes and have above than average svi scores**

In [193]:
contingency_table = pd.crosstab(evi_bh_df.above_bh_average, evi_bh_df.above_evi_average)
contingency_table

above_evi_average,False,True
above_bh_average,,
False,91,6
True,24,53


In [194]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(72.40035499223367),
 np.float64(1.7568234771276802e-17),
 1,
 array([[64.1091954, 32.8908046],
        [50.8908046, 26.1091954]]))

In [195]:
p_value

np.float64(1.7568234771276802e-17)

### **P-value is 0.0000 (extremely small) and chi-squiare is 72.40. We can refute the null hypothesis. We can say we are confident that high "black+hispanic percetage" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high BH pct and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" zipcodes.**

In [196]:
a = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == True)].shape[0]
b = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == False)].shape[0]
c = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == True)].shape[0]
d = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == False)].shape[0]

In [197]:
observed = np.array([[a, b], [c, d]])
observed

array([[53, 24],
       [ 6, 91]])

In [198]:
odds_ratio = (a * d) / (b * c)
odds_ratio

33.49305555555556

**This means, technically, zipcodes with above average b+h pct had 33.4931 times higher odds of having above average eviction rates compared to low-bh-pct zipcodes in nyc.** This high odds may be due to all high bh pct zipcodes had high evictions, and all high eviction zipcodes had high bh pct from the sample.

In [200]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value
# 72.400 is a strong deviation from expected undert the null hypothesis

(np.float64(72.40035499223367), np.float64(1.7568234771276802e-17))